In [55]:
%matplotlib inline
import numpy as np
import pandas as pd
from time import time
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [56]:
%%time
# create a variable for the 20newsgroups dataset training set
newsgroups_train = fetch_20newsgroups(subset='train')

CPU times: user 309 ms, sys: 128 ms, total: 437 ms
Wall time: 632 ms


In [57]:
type(newsgroups_train)

sklearn.utils.Bunch

# pyLDAvis Analysis

In [58]:
# new variable for LDA analysis, removing meta information and pesky footers
news = fetch_20newsgroups(subset='train', shuffle=True, random_state=28,
                remove=('headers', 'footers', 'quotes'))
news_raw = news.data
print(len(news_raw))

11314


In [59]:
type(news_raw)

list

In [60]:
# shows what variables are associated with an object
dir(news)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [61]:
# The publications 
news.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [62]:
%%time
## create a limited set of categories of the 20 for analysis
categories = ['misc.forsale', 'talk.politics.mideast', 
              'sci.space', 'rec.sport.baseball', 'comp.graphics']

# create a new variable for the reduced dataset
# to 4 categories for faster analysis testing
news_reduced = fetch_20newsgroups(subset='train', shuffle=True, random_state=28,
                remove=('headers', 'footers', 'quotes'), categories=categories)

CPU times: user 1.97 s, sys: 155 ms, total: 2.13 s
Wall time: 2.69 s


In [63]:
type(news_reduced)

sklearn.utils.Bunch

In [64]:
# no need to reduce the news to 5 categories
len(news_reduced)

5

## Vectorizing the documents

Count Vectorizer:

In [68]:
# sklearn turns a collection of text documents to a matrix of token counts.
# This allows sentences or small paragraphs to become rows.
# TF vecotrizer
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                preprocessor=None,
                                binary=False,
                                # gets ride of a lot of bad characters
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10,
                                encoding='utf-8',)
# fit the model
dtm_tf = tf_vectorizer.fit_transform(news_raw)
# print the shape
print(dtm_tf.shape)

# different token patterns i tried:
# (?u)\\b\\w\\w+\\b
# OG: \b[^\d\W]+\b/g
# "[^a-zA-Z' ]+"
# ^([a-zA-Z]+|\d+|\W)$
# [ \t\n\r\f\v]

(11314, 9144)


In [69]:
news_raw[4]

'I have the following items for sale.  The highest bid for each to arrive\nin my email box by 5:00 pm EDT Wednesday April 21, 1993 gets the item.\n\n1] Skillcraft Senior Chemlab Set 4581\n        Safe for Ages 10 and Up\n        Used little\n        25 bottles of different Chemicals, Plastic Balance, Alcohol Lamp,\n        Test Tube, Litmus Paper\n        Manual with "Over 1100 Experiments"\n        $4 shipping will be added to your bid\n\n2] Eye of the Beholder II\n        Fun game for the IBM PC\n        In original box, with original media and Manual\n        $3 shipping will be added to your bid\n\n3] Clue Book for Eye of the Beholder II\n\tSolve your dilemmas in a hurry and find all the loot\n\t$1 shipping will be added to your bid\n\tFree shipping on this item if you bid highest on this and item 2].\n\n4] Ethernet Transceiver (ST-500 With LanView (AUI to 50 Ohm coaxial))\n        Works fine\n        Has nifty blinking leds for send/receive/collision/power, etc.\n        Built-in 

In [70]:
# news_raw is a list 
type(news_raw)

list

documentation for Count vectorizer: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

TF-IDF Vectorizer:

In [71]:
# tf-idf vectorizer
# Converts a collection of raw documents to a matrix of TF-IDF features.
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
# fit the model 
dtm_tfidf = tfidf_vectorizer.fit_transform(news_raw)
# print shape
print(dtm_tfidf.shape)

/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1577: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(11314, 9144)


documentation for tf-idf vectorizer: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

# Count Vectorizer Document Visualization

In [87]:
# for TF DTM
lda_tf20 = LatentDirichletAllocation(n_topics=20, random_state=36)
lda_tf20.fit(dtm_tf)

lda_tf10 = LatentDirichletAllocation(n_topics=10, random_state=36)
lda_tf10.fit(dtm_tf)

lda_tf5 = LatentDirichletAllocation(n_topics=5, random_state=36)
lda_tf5.fit(dtm_tf)


/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=5, perp_tol=0.1,
             random_state=36, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

# 20 Topics

In [85]:
#call the model
pyLDAvis.sklearn.prepare(lda_tf20, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
16     11.560750        1       1 -0.088189 -0.102520
18      9.776855        1       2 -0.112204 -0.113567
19      8.657806        1       3 -0.035802 -0.101489
17      8.204880        1       4 -0.093021  0.221173
13      6.987598        1       5 -0.111640  0.109714
8       5.855654        1       6 -0.003635  0.204470
11      5.462843        1       7 -0.100227  0.060092
4       5.169523        1       8 -0.047420 -0.174046
1       4.874125        1       9 -0.018593  0.078920
0       4.671933        1      10 -0.050522 -0.006050
2       4.632234        1      11 -0.122231  0.125331
6       4.268317        1      12 -0.041961 -0.192297
9       3.747677        1      13  0.064138 -0.155481
3       3.608222        1      14 -0.067515  0.064560
15      3.114801        1      15 -0.062761  0.064085
14      2.574073        1      16  0.052833  0.008564
10      2.563956        1      17  0.127836 -0.099858
12      1.900330        1      18  0.004469 -0.107838
5       1.434180        1      19  0.331164 -0.042104
7       0.934244        1      20  0.375281  0.158338, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
5016  Default  4579.000000          max  4579.000000  30.0000  30.0000
2653  Default  2428.000000          edu  2428.000000  29.0000  29.0000
3523  Default  1947.000000          god  1947.000000  28.0000  28.0000
7677  Default  1275.000000        space  1275.000000  27.0000  27.0000
8982  Default  1229.000000      windows  1229.000000  26.0000  26.0000
4497  Default  1195.000000          key  1195.000000  25.0000  25.0000
3153  Default  1712.000000         file  1712.000000  24.0000  24.0000
1157  Default   788.000000          car   788.000000  23.0000  23.0000
804   Default   456.000000          bhj   456.000000  22.0000  22.0000
1506  Default  1429.000000          com  1429.000000  21.0000  21.0000
2560  Default  1101.000000        drive  1101.000000  20.0000  20.0000
3497  Default   433.000000          giz   433.000000  19.0000  19.0000
5924  Default  4071.000000       people  4071.000000  18.0000  18.0000
3639  Default   704.000000          gun   704.000000  17.0000  17.0000
7141  Default  1718.000000         said  1718.000000  16.0000  16.0000
8675  Default  3138.000000          use  3138.000000  15.0000  15.0000
3405  Default   811.000000         game   811.000000  14.0000  14.0000
4396  Default   873.000000        jesus   873.000000  13.0000  13.0000
8167  Default   758.000000         team   758.000000  12.0000  12.0000
4468  Default  3785.000000         just  3785.000000  11.0000  11.0000
2517  Default   709.000000          dos   709.000000  10.0000  10.0000
4543  Default  3483.000000         know  3483.000000   9.0000   9.0000
4895  Default  1196.000000         mail  1196.000000   8.0000   8.0000
7250  Default   630.000000         scsi   630.000000   7.0000   7.0000
2509  Default  3919.000000          don  3919.000000   6.0000   6.0000
9111  Default  1429.000000         year  1429.000000   5.0000   5.0000
6362  Default  1372.000000      program  1372.000000   4.0000   4.0000
3156  Default   821.000000        files   821.000000   3.0000   3.0000
3551  Default  1216.000000   government  1216.000000   2.0000   2.0000
1161  Default   779.000000         card   779.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
5528  Topic20   111.899615         nrhj   112.836456   4.6649  -4.2245
9087  Topic20    81.939879         wwiz    82.876720   4.6618  -4.5362
5545  Topic20    77.945247          nuy    78.882088   4.6612  -4.5861
3481  Topic20    77.945247          ghj    78.882088   4.6612  -4.5861
805   Topic20    60.968064         bhjn    61.904904   4.6579  -4.8318
5325  Topic20    43.990880          mtm    44.927720   4.6521  -5.1582


# 10 Topics

In [88]:
pyLDAvis.sklearn.prepare(lda_tf10, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      15.496871        1       1  0.141031  0.027042
9      14.600841        1       2  0.084689 -0.065014
3      12.602951        1       3 -0.121044  0.080294
8      10.501119        1       4 -0.240718  0.009917
0      10.467467        1       5  0.063285 -0.114917
2      10.143905        1       6 -0.173016  0.038355
1       8.588988        1       7  0.026019 -0.074793
5       7.058704        1       8  0.126729  0.272968
6       6.290237        1       9 -0.043073 -0.104488
7       4.248917        1      10  0.136097 -0.069365, topic_info=     Category         Freq            Term        Total  loglift  logprob
term                                                                     
5016  Default  4443.000000             max  4443.000000  30.0000  30.0000
2653  Default  2402.000000             edu  2402.000000  29.0000  29.0000
3523  Default  1954.000000             god  1954.000000  28.0000  28.0000
4497  Default  1198.000000             key  1198.000000  27.0000  27.0000
7677  Default  1265.000000           space  1265.000000  26.0000  26.0000
3153  Default  1721.000000            file  1721.000000  25.0000  25.0000
8982  Default  1226.000000         windows  1226.000000  24.0000  24.0000
3405  Default   809.000000            game   809.000000  23.0000  23.0000
2560  Default  1112.000000           drive  1112.000000  22.0000  22.0000
8167  Default   755.000000            team   755.000000  21.0000  21.0000
1506  Default  1418.000000             com  1418.000000  20.0000  20.0000
4396  Default   875.000000           jesus   875.000000  19.0000  19.0000
6263  Default   643.000000       president   643.000000  18.0000  18.0000
8675  Default  3143.000000             use  3143.000000  17.0000  17.0000
6362  Default  1372.000000         program  1372.000000  16.0000  16.0000
5924  Default  4081.000000          people  4081.000000  15.0000  15.0000
3551  Default  1225.000000      government  1225.000000  14.0000  14.0000
8278  Default  2979.000000           think  2979.000000  13.0000  13.0000
2747  Default   542.000000      encryption   542.000000  12.0000  12.0000
1322  Default   694.000000            chip   694.000000  11.0000  11.0000
3639  Default   718.000000             gun   718.000000  10.0000  10.0000
1161  Default   787.000000            card   787.000000   9.0000   9.0000
804   Default   442.000000             bhj   442.000000   8.0000   8.0000
3406  Default   551.000000           games   551.000000   7.0000   7.0000
8981  Default   778.000000          window   778.000000   6.0000   6.0000
3497  Default   420.000000             giz   420.000000   5.0000   5.0000
627   Default  1292.000000       available  1292.000000   4.0000   4.0000
3528  Default  1393.000000           going  1393.000000   3.0000   3.0000
7250  Default   638.000000            scsi   638.000000   2.0000   2.0000
9111  Default  1431.000000            year  1431.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
2170  Topic10    40.221746         deficit    46.634788   3.0106  -6.7624
3261  Topic10    44.336509           foods    53.459095   2.9714  -6.6650
3388  Topic10    82.897698           funds   112.065482   2.8570  -6.0392
8156  Topic10    74.920719           taxes    99.694557   2.8728  -6.1404
7278  Topic10    99.596073       secretary   149.518427   2.7522  -5.8557
1423  Topic10   158.303996         clinton   274.480029   2.6081  -5.3923
822   Topic10    98.324772         billion   148.106149   2.7489  -5.8685
1029  Topic10    82.754845          budget   122.008104   2.7703  -6.0409
5249  Topic10   242.472774           money   552.734125   2.3345  -4.9659
219   Topic10   185.055745             air   387.095482   2.4205  -5.2362
7333  Topic10    66.987138          senior    91.565322   2.8460  -6.2523
2639  Topic10    68.074577         economy    96.8815

# 5 Topics

In [89]:
pyLDAvis.sklearn.prepare(lda_tf5, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      27.520152        1       1 -0.110282  0.011490
2      19.610738        1       2  0.115725 -0.110689
1      19.252049        1       3 -0.077647  0.135248
4      18.930851        1       4 -0.165473 -0.082219
3      14.686210        1       5  0.237677  0.046169, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
5016  Default  4433.000000          max  4433.000000  30.0000  30.0000
3523  Default  1900.000000          god  1900.000000  29.0000  29.0000
2653  Default  2367.000000          edu  2367.000000  28.0000  28.0000
4497  Default  1185.000000          key  1185.000000  27.0000  27.0000
8982  Default  1245.000000      windows  1245.000000  26.0000  26.0000
3153  Default  1734.000000         file  1734.000000  25.0000  25.0000
4396  Default   851.000000        jesus   851.000000  24.0000  24.0000
1506  Default  1400.000000          com  1400.000000  23.0000  23.0000
7677  Default  1284.000000        space  1284.000000  22.0000  22.0000
4895  Default  1174.000000         mail  1174.000000  21.0000  21.0000
627   Default  1282.000000    available  1282.000000  20.0000  20.0000
1322  Default   684.000000         chip   684.000000  19.0000  19.0000
8675  Default  3166.000000          use  3166.000000  18.0000  18.0000
2560  Default  1127.000000        drive  1127.000000  17.0000  17.0000
3405  Default   798.000000         game   798.000000  16.0000  16.0000
1157  Default   801.000000          car   801.000000  15.0000  15.0000
3551  Default  1238.000000   government  1238.000000  14.0000  14.0000
7629  Default   950.000000     software   950.000000  13.0000  13.0000
3365  Default   738.000000          ftp   738.000000  12.0000  12.0000
457   Default   614.000000     armenian   614.000000  11.0000  11.0000
8981  Default   790.000000       window   790.000000  10.0000  10.0000
2077  Default  1225.000000         data  1225.000000   9.0000   9.0000
2747  Default   535.000000   encryption   535.000000   8.0000   8.0000
8167  Default   746.000000         team   746.000000   7.0000   7.0000
8252  Default  1255.000000       thanks  1255.000000   6.0000   6.0000
5924  Default  4077.000000       people  4077.000000   5.0000   5.0000
6460  Default   528.000000          pub   528.000000   4.0000   4.0000
4134  Default  1460.000000  information  1460.000000   3.0000   3.0000
2517  Default   716.000000          dos   716.000000   2.0000   2.0000
3639  Default   716.000000          gun   716.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
348    Topic5   389.794379    anonymous   419.711882   1.8443  -5.7314
5970   Topic5   136.376777          pgp   141.300014   1.8828  -6.7816
6315   Topic5   315.020298      privacy   353.195603   1.8039  -5.9444
5448   Topic5    90.810971     networks    92.866220   1.8959  -7.1883
7441   Topic5   226.248220     shipping   248.278950   1.8253  -6.2754
2653   Topic5  1800.643055          edu  2367.999568   1.6444  -4.2012
3099   Topic5   283.952085          fax   320.377701   1.7976  -6.0483
3940   Topic5   325.985908          ibm   375.014567   1.7782  -5.9102
2689   Topic5   201.996710   electronic   222.085076   1.8235  -6.3888
8143   Topic5   213.182854          tar   239.783820   1.8007  -6.3349
5989   Topic5   399.357367        phone   485.835323   1.7222  -5.7072
3365   Topic5   575.421151          ftp   738.022388   1.6694  -5.3420
1506   Topic5   946.827087          com  1400.914186   1.5265  -4.8439
4895   Topic5   803.576673         mail  1174.131904   1.5390  -5.0080
627    Topic5   854.633927    available  1282.766336   1.5122  -4.9464
7146   Topic5   278.121619         sale   334.438855   1.7339  -6.0690
2704   Topic5   490.589063        email   743.824047   1.5021  -5.5015
1629   Topic5   493.764542   

# TF-IDF Document Visualization 

In [90]:
# for TFIDF DTM
# 20 topics
lda_tfidf20 = LatentDirichletAllocation(n_topics=20, random_state=36)
lda_tfidf20.fit(dtm_tfidf)

# 10 topics
lda_tfidf10 = LatentDirichletAllocation(n_topics=10, random_state=36)
lda_tfidf10.fit(dtm_tfidf)

# 5 topics
lda_tfidf5 = LatentDirichletAllocation(n_topics=5, random_state=36)
lda_tfidf5.fit(dtm_tfidf)

/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=5, perp_tol=0.1,
             random_state=36, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

## 20 Topics

In [76]:
pyLDAvis.sklearn.prepare(lda_tfidf20, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      26.405277        1       1  0.161878 -0.054135
11     16.337229        1       2  0.140175 -0.200979
19     16.282287        1       3  0.137323 -0.045532
17     10.432660        1       4  0.276067  0.100075
13      9.298429        1       5  0.263575  0.098887
0       3.089486        1       6 -0.089298  0.013189
5       2.191262        1       7 -0.078299  0.148275
9       2.134644        1       8 -0.070727  0.108596
7       1.599038        1       9 -0.060484 -0.009457
12      1.417796        1      10 -0.073795  0.038723
18      1.344830        1      11 -0.085814 -0.028623
2       1.339789        1      12 -0.059677 -0.028504
8       1.074091        1      13 -0.065380 -0.015618
16      1.063310        1      14 -0.052168 -0.017108
10      1.027331        1      15 -0.055773 -0.019740
3       1.025695        1      16 -0.059730 -0.017717
1       1.021487        1      17 -0.056521 -0.019214
15      1.016122        1      18 -0.057334 -0.016676
14      0.965860        1      19 -0.062099 -0.018682
6       0.933379        1      20 -0.051919 -0.015758, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
8982  Default   77.000000      windows   77.000000  30.0000  30.0000
3405  Default   51.000000         game   51.000000  29.0000  29.0000
1506  Default   63.000000          com   63.000000  28.0000  28.0000
8167  Default   43.000000         team   43.000000  27.0000  27.0000
2560  Default   69.000000        drive   69.000000  26.0000  26.0000
3523  Default   83.000000          god   83.000000  25.0000  25.0000
9111  Default   66.000000         year   66.000000  24.0000  24.0000
2653  Default   89.000000          edu   89.000000  23.0000  23.0000
8252  Default  111.000000       thanks  111.000000  22.0000  22.0000
3406  Default   40.000000        games   40.000000  21.0000  21.0000
7677  Default   47.000000        space   47.000000  20.0000  20.0000
120   Default   39.000000      address   39.000000  19.0000  19.0000
1161  Default   58.000000         card   58.000000  18.0000  18.0000
3153  Default   54.000000         file   54.000000  17.0000  17.0000
8235  Default   25.000000         test   25.000000  16.0000  16.0000
457   Default   18.000000     armenian   18.000000  15.0000  15.0000
3837  Default   23.000000       hockey   23.000000  14.0000  14.0000
6076  Default   31.000000         play   31.000000  13.0000  13.0000
6329  Default   41.000000     problems   41.000000  12.0000  12.0000
458   Default   16.000000    armenians   16.000000  11.0000  11.0000
7657  Default   28.000000         soon   28.000000  10.0000  10.0000
5323  Default   16.000000          msg   16.000000   9.0000   9.0000
7141  Default   64.000000         said   64.000000   8.0000   8.0000
8675  Default  112.000000          use  112.000000   7.0000   7.0000
8532  Default   16.000000      turkish   16.000000   6.0000   6.0000
8981  Default   42.000000       window   42.000000   5.0000   5.0000
7250  Default   33.000000         scsi   33.000000   4.0000   4.0000
3776  Default   24.000000        hello   24.000000   3.0000   3.0000
3156  Default   40.000000        files   40.000000   2.0000   2.0000
7266  Default   24.000000       season   24.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
5913  Topic20    0.999280          pem    2.156330   3.9050  -6.2880
3172  Topic20    0.718963  financially    1.569060   3.8937  -6.6173
2808  Topic20    0.821709          eos    1.885244   3.8437  -6.4837
5236  Topic20    0.917923      modular    2.159464   3.8186  -6.3730
779   Topic20    0.692529     bellcore    1.763025   3.7397  -6.6547
4631  Topic20    1.132306       laying    3.061009   3.6796  -6.1631
2373  Topic20    0.680393    directive    1.879733   3.6579  -6.6724
6508  Top

# 10 Topics

In [91]:
pyLDAvis.sklearn.prepare(lda_tfidf10, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      32.380367        1       1 -0.024728 -0.089898
7      23.281143        1       2 -0.053599 -0.051552
8      16.182689        1       3 -0.199842  0.018058
2      10.796458        1       4 -0.201705  0.068375
0       5.610185        1       5  0.044910 -0.186607
6       3.098683        1       6  0.079090  0.099325
5       2.481570        1       7  0.099032  0.045979
9       2.159974        1       8  0.089082  0.031843
1       2.095725        1       9  0.094577  0.038815
3       1.913206        1      10  0.073183  0.025661, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
4497  Default   52.000000          key   52.000000  30.0000  30.0000
2653  Default   89.000000          edu   89.000000  29.0000  29.0000
8982  Default   77.000000      windows   77.000000  28.0000  28.0000
8167  Default   42.000000         team   42.000000  27.0000  27.0000
3405  Default   50.000000         game   50.000000  26.0000  26.0000
1322  Default   42.000000         chip   42.000000  25.0000  25.0000
8252  Default  111.000000       thanks  111.000000  24.0000  24.0000
2747  Default   26.000000   encryption   26.000000  23.0000  23.0000
9111  Default   65.000000         year   65.000000  22.0000  22.0000
3153  Default   54.000000         file   54.000000  21.0000  21.0000
3551  Default   51.000000   government   51.000000  20.0000  20.0000
3406  Default   39.000000        games   39.000000  19.0000  19.0000
7657  Default   28.000000         soon   28.000000  18.0000  18.0000
1425  Default   25.000000      clipper   25.000000  17.0000  17.0000
1506  Default   62.000000          com   62.000000  16.0000  16.0000
8675  Default  113.000000          use  113.000000  15.0000  15.0000
3639  Default   32.000000          gun   32.000000  14.0000  14.0000
3156  Default   40.000000        files   40.000000  13.0000  13.0000
4500  Default   23.000000         keys   23.000000  12.0000  12.0000
1161  Default   59.000000         card   59.000000  11.0000  11.0000
2560  Default   70.000000        drive   70.000000  10.0000  10.0000
8981  Default   42.000000       window   42.000000   9.0000   9.0000
6362  Default   54.000000      program   54.000000   8.0000   8.0000
6080  Default   26.000000      players   26.000000   7.0000   7.0000
3523  Default   83.000000          god   83.000000   6.0000   6.0000
7266  Default   24.000000       season   24.000000   5.0000   5.0000
8235  Default   25.000000         test   25.000000   4.0000   4.0000
6038  Default   16.000000         pitt   16.000000   3.0000   3.0000
3538  Default   16.000000       gordon   16.000000   2.0000   2.0000
3365  Default   32.000000          ftp   32.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
3728  Topic10    2.696200         hawk    4.114869   3.5336  -6.0132
9094  Topic10    1.870177       ximage    2.890691   3.5209  -6.3790
2857  Topic10    1.823281       ethics    2.850704   3.5095  -6.4044
4843  Topic10    1.964111   louisville    3.089663   3.5034  -6.3300
3389  Topic10    1.511109        funet    2.379652   3.5023  -6.5922
146   Topic10    1.912245       adress    3.019498   3.4996  -6.3568
2278  Topic10    1.507368  destination    2.386279   3.4970  -6.5947
1298  Topic10    2.951744        cheat    4.706074   3.4899  -5.9227
5778  Topic10    1.336166      packing    2.229736   3.4443  -6.7152
4327  Topic10    1.892747      ireland    3.234179   3.4206  -6.3670
2028  Topic10    1.622555       curves    2.776672   3.4191  -6.5210
8692  Topic10    1.496815         util    2.603624   3.4028  -6.6017
5465  Topic10    1.543289          nic    2.686550   3.4020  -6.5711
4719  Topic10    1.423550      licence    2.506513   3.3907  -6.6519
3813  Topic10    2.385950      hillary    4.226210   3.3847  -6.13

# 5 Topics

In [92]:
pyLDAvis.sklearn.prepare(lda_tfidf5, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      56.418530        1       1  0.024697  0.071142
2      24.570320        1       2 -0.153494  0.028205
0       7.918164        1       3  0.112647  0.069135
3       6.387904        1       4 -0.071717 -0.060085
1       4.705081        1       5  0.087868 -0.108397, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
8252  Default  112.000000      thanks  112.000000  30.0000  30.0000
7677  Default   49.000000       space   49.000000  29.0000  29.0000
8982  Default   80.000000     windows   80.000000  28.0000  28.0000
2653  Default   90.000000         edu   90.000000  27.0000  27.0000
4497  Default   49.000000         key   49.000000  26.0000  26.0000
4895  Default   69.000000        mail   69.000000  25.0000  25.0000
1161  Default   59.000000        card   59.000000  24.0000  24.0000
7146  Default   35.000000        sale   35.000000  23.0000  23.0000
2560  Default   70.000000       drive   70.000000  22.0000  22.0000
2704  Default   50.000000       email   50.000000  21.0000  21.0000
4355  Default   30.000000      israel   30.000000  20.0000  20.0000
6285  Default   41.000000       price   41.000000  19.0000  19.0000
5452  Default   99.000000         new   99.000000  18.0000  18.0000
3551  Default   50.000000  government   50.000000  17.0000  17.0000
120   Default   39.000000     address   39.000000  16.0000  16.0000
5615  Default   31.000000       offer   31.000000  15.0000  15.0000
4244  Default   44.000000  interested   44.000000  14.0000  14.0000
3153  Default   55.000000        file   55.000000  13.0000  13.0000
7657  Default   28.000000        soon   28.000000  12.0000  12.0000
7441  Default   25.000000    shipping   25.000000  11.0000  11.0000
1506  Default   63.000000         com   63.000000  10.0000  10.0000
2493  Default  129.000000        does  129.000000   9.0000   9.0000
2747  Default   24.000000  encryption   24.000000   8.0000   8.0000
8777  Default   37.000000       video   37.000000   7.0000   7.0000
2517  Default   43.000000         dos   43.000000   6.0000   6.0000
4821  Default   58.000000     looking   58.000000   5.0000   5.0000
4874  Default   39.000000         mac   39.000000   4.0000   4.0000
8675  Default  114.000000         use  114.000000   3.0000   3.0000
5372  Default   22.000000        nasa   22.000000   2.0000   2.0000
7629  Default   48.000000    software   48.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
7678   Topic5    7.388783  spacecraft    8.068018   2.9686  -5.9050
8050   Topic5   13.928154   surrender   15.636885   2.9408  -5.2710
7172   Topic5    7.716644   satellite    8.739524   2.9321  -5.8615
354    Topic5    6.131953     antenna    7.006317   2.9232  -6.0914
4977   Topic5    4.792592        mars    5.476719   2.9231  -6.3378
4464   Topic5    4.502885     jupiter    5.182808   2.9159  -6.4002
674    Topic5    4.213249      baerga    4.901448   2.9052  -6.4667
5689   Topic5    4.144257     orbital    4.824373   2.9046  -6.4832
7173   Topic5    3.998698  satellites    4.673553   2.9006  -6.5189
1198   Topic5    3.859620     catbyte    4.532929   2.8957  -6.5543
2582   Topic5    3.859620     dtmedin    4.532929   2.8957  -6.5543
4140   Topic5    3.840143        ingr    4.514171   2.8948  -6.5594
1013   Topic5    3.825978       bronx    4.504904   2.8932  -6.5631
4933   Topic5    3.830450   manhattan    4.511203   2.8929  -6.5619
8654   Topic5    3.868120       upenn    4.572749   2.8892  -6.5521
819    Topic5    3.662539       biker    4.355775   2.8832  -6.6068
7768   Topic5    3.459924        ssto    4.134581   2.8784  -6.6637
890    Topic5    3.394409       bobbe    4.067397   2.8757  -6.6828
4614   Topic5   11.299090      launch   13.631521   2.8689  -5.4802
4863   Topic5    7.890700       lunar    9.

## Insert segue to model evaluation

## Intro Jensen-Shannon?

## Jensen-Shannon Divergence & PCA  -- NEEDS MORE WORK!!

In [ ]:
lda_tfidf20, dtm_tfidf, tfidf_vectorizer

In [98]:
type(lda_tfidf20)

sklearn.decomposition.online_lda.LatentDirichletAllocation

In [99]:
type(news_raw)

list

In [115]:
from scipy.spatial.distance import pdist, squareform
from scipy.stats import entropy

def _jensen_shannon(_P, _Q):
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

def js_PCoA(distributions):
    """Dimension reduction via Jensen-Shannon Divergence & Principal Coordinate Analysis
    (aka Classical Multidimensional Scaling)

    Parameters
    ----------
    distributions : array-like, shape (`n_dists`, `k`)
        Matrix of distributions probabilities.

    Returns
    -------
    pcoa : array, shape (`n_dists`, 2)
    """
    dist_matrix = squareform(pdist(distributions, metric=_jensen_shannon))
    return _pcoa(dist_matrix)

In [119]:
js = np.array(news_raw)

In [120]:
type(js)

numpy.ndarray

In [118]:
js = js_PCoA(news_raw)

ValueError: A 2-dimensional array must be passed.

In [ ]:
jensen-shannon divergence

In [109]:
pyLDAvis.sklearn.prepare(lda_tfidf5, dtm_tfidf, tfidf_vectorizer, mds='pcoa')

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      56.418530        1       1  0.024697  0.071142
2      24.570320        1       2 -0.153494  0.028205
0       7.918164        1       3  0.112647  0.069135
3       6.387904        1       4 -0.071717 -0.060085
1       4.705081        1       5  0.087868 -0.108397, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
8252  Default  112.000000      thanks  112.000000  30.0000  30.0000
7677  Default   49.000000       space   49.000000  29.0000  29.0000
8982  Default   80.000000     windows   80.000000  28.0000  28.0000
2653  Default   90.000000         edu   90.000000  27.0000  27.0000
4497  Default   49.000000         key   49.000000  26.0000  26.0000
4895  Default   69.000000        mail   69.000000  25.0000  25.0000
1161  Default   59.000000        card   59.000000  24.0000  24.0000
7146  Default   35.000000        sale   35.000000  23.0000  23.0000
2560  Default   70.000000       drive   70.000000  22.0000  22.0000
2704  Default   50.000000       email   50.000000  21.0000  21.0000
4355  Default   30.000000      israel   30.000000  20.0000  20.0000
6285  Default   41.000000       price   41.000000  19.0000  19.0000
5452  Default   99.000000         new   99.000000  18.0000  18.0000
3551  Default   50.000000  government   50.000000  17.0000  17.0000
120   Default   39.000000     address   39.000000  16.0000  16.0000
5615  Default   31.000000       offer   31.000000  15.0000  15.0000
4244  Default   44.000000  interested   44.000000  14.0000  14.0000
3153  Default   55.000000        file   55.000000  13.0000  13.0000
7657  Default   28.000000        soon   28.000000  12.0000  12.0000
7441  Default   25.000000    shipping   25.000000  11.0000  11.0000
1506  Default   63.000000         com   63.000000  10.0000  10.0000
2493  Default  129.000000        does  129.000000   9.0000   9.0000
2747  Default   24.000000  encryption   24.000000   8.0000   8.0000
8777  Default   37.000000       video   37.000000   7.0000   7.0000
2517  Default   43.000000         dos   43.000000   6.0000   6.0000
4821  Default   58.000000     looking   58.000000   5.0000   5.0000
4874  Default   39.000000         mac   39.000000   4.0000   4.0000
8675  Default  114.000000         use  114.000000   3.0000   3.0000
5372  Default   22.000000        nasa   22.000000   2.0000   2.0000
7629  Default   48.000000    software   48.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
7678   Topic5    7.388783  spacecraft    8.068018   2.9686  -5.9050
8050   Topic5   13.928154   surrender   15.636885   2.9408  -5.2710
7172   Topic5    7.716644   satellite    8.739524   2.9321  -5.8615
354    Topic5    6.131953     antenna    7.006317   2.9232  -6.0914
4977   Topic5    4.792592        mars    5.476719   2.9231  -6.3378
4464   Topic5    4.502885     jupiter    5.182808   2.9159  -6.4002
674    Topic5    4.213249      baerga    4.901448   2.9052  -6.4667
5689   Topic5    4.144257     orbital    4.824373   2.9046  -6.4832
7173   Topic5    3.998698  satellites    4.673553   2.9006  -6.5189
1198   Topic5    3.859620     catbyte    4.532929   2.8957  -6.5543
2582   Topic5    3.859620     dtmedin    4.532929   2.8957  -6.5543
4140   Topic5    3.840143        ingr    4.514171   2.8948  -6.5594
1013   Topic5    3.825978       bronx    4.504904   2.8932  -6.5631
4933   Topic5    3.830450   manhattan    4.511203   2.8929  -6.5619
8654   Topic5    3.868120       upenn    4.572749   2.8892  -6.5521
819    Topic5    3.662539       biker    4.355775   2.8832  -6.6068
7768   Topic5    3.459924        ssto    4.134581   2.8784  -6.6637
890    Topic5    3.394409       bobbe    4.067397   2.8757  -6.6828
4614   Topic5   11.299090      launch   13.631521   2.8689  -5.4802
4863   Topic5    7.890700       lunar    9.

A function that takes topic_term_dists as an input and outputs a n_topics by 2 distance matrix. The output approximates the distance between topics. See js_PCoA() for details on the default function. A string representation currently accepts pcoa (or upper case variant), mmds (or upper case variant) and tsne (or upper case variant), if sklearn package is installed for the latter two.

In [107]:
pyLDAvis.js_PCoA()

ValueError: A 2-dimensional array must be passed.

## Traditional NLP??

In [25]:
## import 20newsgorups as a list?

In [4]:
%%time
from pprint import pprint
# view the dictionary keys
pprint(list(newsgroups_train))

['data', 'filenames', 'target_names', 'target', 'DESCR']
CPU times: user 576 µs, sys: 1.19 ms, total: 1.77 ms
Wall time: 990 µs


In [5]:
%%time
# understanding the length and shape of the file
print(newsgroups_train.filenames.shape)
print(newsgroups_train.target.shape)
print(newsgroups_train.target[:10])

(11314,)
(11314,)
[ 7  4  4  1 14 16 13  3  2  4]
CPU times: user 608 µs, sys: 878 µs, total: 1.49 ms
Wall time: 683 µs


In [59]:
%%time
#from collections import Counter
from sklearn.datasets import fetch_20newsgroups
import re

# define text cleaner function for the dataset
def text_cleaner(line:str):
    '''define the function for a text cleaner to run the news through'''
        # replaces new line with a space
    line = re.sub('\n', ' ', line)
        # removes characters outside of a-z w/ a space
    line = re.sub('[^a-z ]', ' ', line)
        # removes things within a [] bracket and replaces w/ a space
    line = re.sub("[[].*?[]]", ' ', line)
        # removes -- and replaces w/ a space
    line = re.sub(r'--', ' ', line)
        # splits sentences and makes it a list 
    line = ' '.join(line.split())
    return line

# deprecation warning had me remove \ in line 14, "[\[].*?[\]]"
# source: https://github.com/pallets/jinja/issues/646

CPU times: user 12 µs, sys: 1e+03 ns, total: 13 µs
Wall time: 16.9 µs


In [21]:
# not necessary due to tfidf vectorization?

#news_reduced = text_cleaner(news_reduced)

In [13]:
%%time
import spacy
nlp = spacy.load('en')
nlp.max_length = 23065807
news_doc = nlp(" ".join(news_reduced["data"]))

CPU times: user 1min 49s, sys: 1min 2s, total: 2min 52s
Wall time: 4min 8s


In [15]:
%%time
news_sents = [[sent, "data"] for sent in news_doc.sents]

# Create one data frame.
sentences = pd.DataFrame(news_sents)
sentences.head()

CPU times: user 263 ms, sys: 428 ms, total: 691 ms
Wall time: 962 ms


In [6]:
%%time
#from collections import Counter
from sklearn.datasets import fetch_20newsgroups
import re

# define text cleaner function for the dataset
def text_cleaner(line:str):
    '''define the function for a text cleaner to run the news through'''
        # replaces new line with a space
    line = re.sub('\n', ' ', line)
        # removes characters outside of a-z w/ a space
    line = re.sub('[^a-z ]', ' ', line)
        # removes things within a [] bracket and replaces w/ a space
    line = re.sub("[[].*?[]]", ' ', line)
        # removes -- and replaces w/ a space
    line = re.sub(r'--', ' ', line)
        # splits sentences and makes it a list 
    line = ' '.join(line.split())
    return line

# deprecation warning had me remove \ in line 14, "[\[].*?[\]]"
# source: https://github.com/pallets/jinja/issues/646

CPU times: user 14 µs, sys: 1e+03 ns, total: 15 µs
Wall time: 19.1 µs


In [7]:
# saving for pythonic example of a good for loop
#tokens = [token for token in tokens if len(token) > 0]

In [16]:
# all the text is being labeled "data." We need it to say the proper publication
sentences.head()

,0,1
0,"(Yes, ,, the, Phobos, mission, did, return, so...",data
1,"(The, best, I, 've, seen, had, a, surface, res...",data
2,"(By, \n, the, way, ,, the, new, book, entitled...",data
3,"(The, chapter, is, co, -, authored, by, V.I., ...",data
4,"(Do, n't, \n, know, of, any, ftp, sites, with,...",data


In [25]:
%%time
# Mike--> code from our session last week
from typing import List
def bag_of_words_spacy(doc:spacy.tokens.doc.Doc)->List:
    """takes spacy doc, returns list of most common 2000 lemmas"""
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in doc
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

# Mike--> code from our session last week
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features_df(sentences:List, common_words)->pd.DataFrame:
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['label'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(sentences[0]):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [span.lemma_
                 for span in sentence
                 if (
                     not span.is_punct
                     and not span.is_stop
                     and span.lemma_ in common_words
                 )
                ]
        

         #Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 14.3 µs
